In [ ]:
#ANN 10

In [7]:
import tensorflow as tf
import numpy as np
import cv2


In [9]:

# Define the path to the pre-trained model
model_path = r"C:\Users\Avina\ANN practical\ssd_mobilenet_v2_320x320_coco17_tpu-8\saved_model"

# Load the pre-trained model
model = tf.saved_model.load(model_path)

In [10]:
# Define the label map
label_map = {1: 'person', 2: 'car', 3: 'dog'}  # Replace with your label map

In [11]:
path = r"C:\Users\Avina\ANN practical\test.png"

# Read the image using OpenCV
image = cv2.imread(path)

In [12]:
if image is None:
    print("Error: Image not loaded. Check file path or file existence.")
else:
    # Convert image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Prepare the image for the model input
    image_resized = cv2.resize(image_rgb, (320, 320))  # Resize to the input size of the model
    image_tensor = np.expand_dims(image_resized, axis=0)  # Add batch dimension
    image_tensor = tf.convert_to_tensor(image_tensor, dtype=tf.uint8)

    # Perform object detection
    detections = model(image_tensor)
    boxes = detections['detection_boxes'][0].numpy()
    scores = detections['detection_scores'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(int)

    # Set the threshold for detections
    threshold = 0.5
    filtered_boxes = boxes[scores >= threshold]
    filtered_classes = classes[scores >= threshold]


In [13]:
for box, cls in zip(filtered_boxes, filtered_classes):
        ymin, xmin, ymax, xmax = box
        ymin *= image.shape[0]
        xmin *= image.shape[1]
        ymax *= image.shape[0]
        xmax *= image.shape[1]

        label = label_map.get(cls, 'Unknown')

        # Draw bounding box and label on the image
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
        cv2.putText(image, label, (int(xmin), int(ymin) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


In [14]:
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Display the image with bounding boxes
    cv2.imshow('Detected Objects', image_bgr)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

print(f"Boxes detected: {len(filtered_boxes)}")
print(filtered_boxes)


Boxes detected: 2
[[0.03735748 0.15418722 0.9313384  0.52299887]
 [0.46340597 0.03166941 0.96292806 0.74925494]]
